In [1]:
import pandas as pd
# from api_methods import *
from api_methods import get_chanel_stats, get_video_ids, get_video_comments

from googleapiclient.discovery import build
from IPython.display import JSON

from config.config import YOUTUBE_API_KEY

In [2]:
# Get credentials and create an API client
api_service_name = "youtube"
api_version = "v3"

youtube = build(
    api_service_name, api_version, developerKey=YOUTUBE_API_KEY)

In [3]:
# Channels to analyze
channel_ids = [
    "UCYO_jab_esuFRV4b17AJtAw",
    # Can add more channels
]

In [4]:
channel_stats = get_chanel_stats(youtube, channel_ids)

In [5]:
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,3Blue1Brown,5820000,443960488,168,UUYO_jab_esuFRV4b17AJtAw


In [6]:
playlist_id = channel_stats.loc[channel_stats['channelName']== "3Blue1Brown", "playlistId"].iloc[0]

In [7]:
video_idss = get_video_ids(youtube, playlist_id)

In [8]:
len(video_idss)

168

In [10]:
video_idss[:5]

['xIMlJUwB1m8', 'IstyL9vnybg', 'DWnb2UUlqd0', 'U3O1nzSojnc', 'XIW-2ykgVPI']

In [11]:
video_ids = [
    "xIMlJUwB1m8",
]

In [12]:
# Get video comments
video_comments = get_video_comments(youtube, video_ids)
# video_comments to dataframe
video_comments_df = pd.DataFrame(video_comments)

In [13]:
video_comments_df.head()

,0
0,It’s the formula for chaos… I could be mistake...
1,What
2,That&#39;s why no one understands women😂😂
3,So... second opinions and further investigatio...
4,Precision has entered the chat
